In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

In [4]:
train_o = pd.read_csv(r'C:\Projects\House-Prices-XGBoost\data\train.csv', index_col="Id")
test_o = pd.read_csv(r'C:\Projects\House-Prices-XGBoost\data\test.csv', index_col="Id")
trainx = train_o.iloc[:,:-1]
trainy = train_o.iloc[:,-1]

In [5]:
trainx

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                      
1             60       RL         65.0     8450   Pave   NaN      Reg   
2             20       RL         80.0     9600   Pave   NaN      Reg   
3             60       RL         68.0    11250   Pave   NaN      IR1   
4             70       RL         60.0     9550   Pave   NaN      IR1   
5             60       RL         84.0    14260   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1456          60       RL         62.0     7917   Pave   NaN      Reg   
1457          20       RL         85.0    13175   Pave   NaN      Reg   
1458          70       RL         66.0     9042   Pave   NaN      Reg   
1459          20       RL         68.0     9717   Pave   NaN      Reg   
1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
Id                                    ...                                      
1            Lvl    AllPub    Inside  ...           0        0    NaN    NaN   
2            Lvl    AllPub       FR2  ...           0        0    NaN    NaN   
3            Lvl    AllPub    Inside  ...           0        0    NaN    NaN   
4            Lvl    AllPub    Corner  ...           0        0    NaN    NaN   
5            Lvl    AllPub       FR2  ...           0        0    NaN    NaN   
...          ...       ...       ...  ...         ...      ...    ...    ...   
1456         Lvl    AllPub    Inside  ...           0        0    NaN    NaN   
1457         Lvl    AllPub    Inside  ...           0        0    NaN  MnPrv   
1458         Lvl    AllPub    Inside  ...           0        0    NaN  GdPrv   
1459         Lvl    AllPub    Inside  ...           0        0    NaN    NaN   
1460         Lvl    AllPub    Inside  ...           0        0    NaN    NaN   

     MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
Id                                                                 
1            NaN       0       2    2008        WD         Normal  
2            NaN       0       5    2007        WD         Normal  
3            NaN       0       9    2008        WD         Normal  
4            NaN       0       2    2006        WD        Abnorml  
5            NaN       0      12    2008        WD         Normal  
...          ...     ...     ...     ...       ...            ...  
1456         NaN       0       8    2007        WD         Normal  
1457         NaN       0       2    2010        WD         Normal  
1458        Shed    2500       5    2010        WD         Normal  
1459         NaN       0       4    2010        WD         Normal  
1460         NaN       0       6    2008        WD         Normal  

[1460 rows x 79 columns]

In [6]:
cat = [i for i in trainx if not (trainx[i].dtype == "int64" or trainx[i].dtype == "float64")]

In [43]:
def ColRemover(X, columns) -> pd.DataFrame:
    """Remove specified columns""" # Used in a pipeline, to allow for Grid-Searching
    if len(columns) == 0:
        return X
    else:
        return X.drop(columns, axis=1)
    

col_remover = FunctionTransformer(ColRemover)

col_trans = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat)], remainder='passthrough')

xgbreg = XGBRegressor()

pipeline = make_pipeline(col_remover, col_trans, xgbreg)

In [25]:
cross_val_score(pipeline, train_o.iloc[:,:-1], train_o.iloc[:,-1], cv=4).mean()

0.8629268278592424

In [46]:
params = {
    "functiontransformer__kw_args": np.array([{"columns":["LowQualFinSF"]}]), # To be searched during EDA
    "xgbregressor__gamma": np.array([0, 0.01]),
    "xgbregressor__learning_rate": np.array([0.09, 0.095, 1, 1.05]),
    "xgbregressor__n_estimators": np.array([100, 150]),
    "xgbregressor__reg_lambda": np.arange(0.01, 0.1, 0.02)
}

gridsearch = GridSearchCV(pipeline, params, verbose=2, pre_dispatch=4, cv=4)
gridsearch.fit(trainx, trainy)

Fitting 4 folds for each of 80 candidates, totalling 320 fits
[CV] END functiontransformer__kw_args={'columns': ['LowQualFinSF']}, xgbregressor__gamma=0.0, xgbregressor__learning_rate=0.09, xgbregressor__n_estimators=100, xgbregressor__reg_lambda=0.01; total time=   0.4s
[CV] END functiontransformer__kw_args={'columns': ['LowQualFinSF']}, xgbregressor__gamma=0.0, xgbregressor__learning_rate=0.09, xgbregressor__n_estimators=100, xgbregressor__reg_lambda=0.01; total time=   0.3s
[CV] END functiontransformer__kw_args={'columns': ['LowQualFinSF']}, xgbregressor__gamma=0.0, xgbregressor__learning_rate=0.09, xgbregressor__n_estimators=100, xgbregressor__reg_lambda=0.01; total time=   0.3s
[CV] END functiontransformer__kw_args={'columns': ['LowQualFinSF']}, xgbregressor__gamma=0.0, xgbregressor__learning_rate=0.09, xgbregressor__n_estimators=100, xgbregressor__reg_lambda=0.01; total time=   0.4s
[CV] END functiontransformer__kw_args={'columns': ['LowQualFinSF']}, xgbregressor__gamma=0.0, xgbr

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('functiontransformer',
                                        FunctionTransformer(func=<function ColRemover at 0x000000EC731AC9D0>)),
                                       ('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['MSZoning',
                                                                          'Street',
                                                                          'Alley',
                                                                          'LotShape',
                                                                          'LandContour',
                                         

In [47]:
gridsearch.best_params_

{'functiontransformer__kw_args': {'columns': ['LowQualFinSF']},
 'xgbregressor__gamma': 0.0,
 'xgbregressor__learning_rate': 0.09,
 'xgbregressor__n_estimators': 150,
 'xgbregressor__reg_lambda': 0.03}

In [48]:
gridsearch.best_score_

0.8761190002457406